# Project Title
Recognising hate speech on twitter (vs offensive language)

### Data description
The dataset contains 24,783 tweets by the twitter users. -- To be fill --

***Data Dictionary***

-- To be fill --


## Part 1: Importing Dataset

In [580]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [581]:
main_data = pd.read_csv("Data/labeled_data.csv")
main_data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


### Importing the Dataset

In [582]:
main_data.info()
#dataframe information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [583]:
main_data.describe()
# Descriptive statistics


,Unnamed: 0,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,12681.192027,3.243473,0.280515,2.413711,0.549247,1.110277
std,7299.553863,0.883060,0.631851,1.399459,1.113299,0.462089
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,6372.500000,3.000000,0.000000,2.000000,0.000000,1.000000
50%,12703.000000,3.000000,0.000000,3.000000,0.000000,1.000000
75%,18995.500000,3.000000,0.000000,3.000000,0.000000,1.000000
max,25296.000000,9.000000,7.000000,9.000000,9.000000,2.000000


Adding a new column to this dataset as labels which will contain the values as:

1. Hate Speech
2. Offensive Language
3. No Hate and Offensive

Data Visualization


In [584]:
# main_data["labels"] = main_data["class"].map({0: "Hate Speech", 
#                                     1: "Offensive Language", 
#                                     2: "No Hate and Offensive"})
main_data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [585]:
main_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [586]:
# main_data[main_data["labels"]=="Hate Speech"].shape[0]

In [587]:
# main_data[main_data["labels"]=="Offensive Language"].shape[0]

In [588]:
# main_data[main_data["labels"]=="No Hate and Offensive"].shape[0]

In [589]:
main_data["tweet"].iloc[2]

'!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit'

Now we only select the tweet and labels columns for the rest of the task of training a hate speech detection model:

i.e. Selecting only the relevant columns which are tweets and labels

In [590]:
relevant_data=main_data[["tweet", "class"]]
relevant_data.head()

,tweet,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1


In [591]:
#getting the sum of duplicates
relevant_data.duplicated().sum()

0

## Part 2: Data Preprocessing

In [592]:
import re
from nltk.tokenize import word_tokenize      #to divide strings into lists of substrings
from nltk.stem import WordNetLemmatizer      #to link words with similar meanings to one word.
from nltk.corpus import stopwords            #to filterout useless data
import nltk
nltk.download('wordnet')
stopword = set(stopwords.words('english'))
import string

In [593]:
import emoji

def convert_emoji_to_unicode_to_word(tweet): 
    
    emoji_list = re.findall(r'&#\d+;',tweet)
    
    for emoji_str in emoji_list:
        decimal_code_point = int(emoji_str.replace("&#", "").replace(";",""))
        tweet = tweet.replace(emoji_str, emoji.demojize(chr(decimal_code_point)).replace("_"," ")+" ")
    return tweet

In [594]:
def clean(text):
    #Convert the tweet to lowercase
    text = str(text).lower()
    if re.match("&#\d+;", text):
        text = convert_emoji_to_unicode_to_word(text)
        
    if re.match("^[^&][\S]+", text):
        text = re.sub(r'#', ' ', text)
    
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    # text = re.sub(r"\@w+|\#",'',text)
    #remove username from the tweet starting with @
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r"[^\w\s]",'',text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # #remove "rt" from the tweet
    text = re.sub(r'rt ', '', text, flags=re.IGNORECASE)
    # Remove all special characters except for astrisk to preserve the meaning of the tweet
    text = re.sub(r'[^a-z0-9*\s]', '', text)
    tweet_tokens = word_tokenize(text)
    filtered_tweets=[w for w in tweet_tokens if not w in stopword] #removing stopwords
    return " ".join(filtered_tweets)

In [595]:
relevant_data.tweet=relevant_data['tweet'].apply(clean)

C:\Users\never\AppData\Local\Temp\ipykernel_18628\620039408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_data.tweet=relevant_data['tweet'].apply(clean)


In [596]:
#getting the sum of duplicates
relevant_data.duplicated().sum()

856

In [597]:
duplicates = relevant_data[relevant_data.duplicated(keep='first')]
duplicates

,tweet,class
563,hoes aint loyal,1
873,porn android iphone ipad sex xxx desi paki,2
883,porn android iphone ipad sex xxx hairy hairy p...,1
1344,twitter rappers dm trash links,2
1449,way fuck ur bitch,1
...,...,...
24660,hoe,1
24724,give bitch inch rather,1
24748,yo hoe get slayed,1
24749,yo hoe get slayed,1


In [598]:
relevant_data["tweet"].iloc[1007]

'face tears joy face tears joy fucking god bruh birds southridge front house'

In [599]:
# removing duplicate data
new_data = relevant_data.drop_duplicates("tweet")

In [600]:
# to see updated number of columns as duplicates entries are removed
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23880 entries, 0 to 24782
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   23880 non-null  object
 1   class   23880 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 559.7+ KB


In [601]:
lemmatizer=WordNetLemmatizer()
def lemmatizing(data):
    tweet=[lemmatizer.lemmatize(word) for word in data]
    return data

In [602]:
#lemmatizing the processed data
new_data['tweet']=new_data['tweet'].apply(lambda x: lemmatizing(x))

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\never/nltk_data'
    - 'c:\\Users\\never\\anaconda3\\nltk_data'
    - 'c:\\Users\\never\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\never\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\never\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
new_data.head()

,tweet,class
0,woman shouldnt complain cleaning house amp man...,2
1,boy dats coldtyga dwn bad cuffin dat hoe place,1
2,dawg ever fuck bitch stato cry confused shit,1
3,look like tranny,1
4,shit hear might true might faker bitch told ya,1
